In [1]:
import tensorflow as tf 
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import pandas as pd 

2024-04-24 23:58:52.081392: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-24 23:58:52.111183: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 23:58:52.111210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 23:58:52.112047: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-24 23:58:52.116796: I tensorflow/core/platform/cpu_feature_guar

In [4]:
def augment(image, label):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [image.shape[0], image.shape[1]])
  image = (image / 255.0)
  image = tf.image.random_crop(image, size=[image.shape[0], image.shape[1], 3])
  image = tf.image.random_brightness(image, max_delta=0.5)
  return image, label

In [2]:
data = cv.imread("left_click.png")
type(data)

numpy.ndarray

In [5]:
x = augment(data, 0)
type(x[0][0].numpy())

numpy.ndarray

In [6]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

def augment_images(image, num_augmented):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    # Normalize pixel values to the range [0, 1]
    image = np.array(image) / 255.0
    image = image.reshape((1,) + image.shape)

    augmented_images = []
    for _ in range(num_augmented):
        augmented_image = datagen.flow(image, batch_size=1).next()[0]
        augmented_images.append(augmented_image)

    return augmented_images

In [7]:
df = pd.DataFrame(columns=["Images","Label"])
df

,Images,Label


In [8]:
num_augmented = 500
augmented_images = augment_images(data, num_augmented)
for i in range(num_augmented):
    a = augmented_images[i]
    # plt.imshow(a)
    df.loc[len(df.index)] = [a,0]
    # plt.show()

In [9]:
df

,Images,Label
0,"[[[0.09803922, 0.09803922, 0.09803922], [0.098...",0
1,"[[[0.17254902, 0.17254902, 0.17254902], [0.172...",0
2,"[[[0.76024514, 0.76024514, 0.76024514], [0.756...",0
3,"[[[0.023529412, 0.023529412, 0.023529412], [0....",0
4,"[[[0.09207635, 0.09207635, 0.09207635], [0.091...",0
...,...,...
495,"[[[0.10980392, 0.10980392, 0.10980392], [0.109...",0
496,"[[[0.17254902, 0.17254902, 0.17254902], [0.172...",0
497,"[[[0.17254902, 0.17254902, 0.17254902], [0.172...",0
498,"[[[0.057778984, 0.057778984, 0.057778984], [0....",0


In [10]:
data.shape 

(572, 646, 3)

In [11]:
model_arch = ["Conv","max","Conv","max","Conv","max","Flatten","DNN"]
filters_size= 10
kernal = (3,3)
img_shape = data.shape
dense_layer = 10

In [12]:
model = tf.keras.Sequential([])
for i in range(10):
    model.add(tf.keras.layers.Conv2D(filters= 10, kernel_size=(3,3), activation = "relu",input_shape = data.shape ))
    model.add(tf.keras.layers.Conv2D(filters= 10, kernel_size=(3,3), activation = "relu",input_shape = data.shape ))
    
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 570, 644, 10)      280       
                                                                 
 conv2d_1 (Conv2D)           (None, 568, 642, 10)      910       
                                                                 
 conv2d_2 (Conv2D)           (None, 566, 640, 10)      910       
                                                                 
 conv2d_3 (Conv2D)           (None, 564, 638, 10)      910       
                                                                 
 conv2d_4 (Conv2D)           (None, 562, 636, 10)      910       
                                                                 
 conv2d_5 (Conv2D)           (None, 560, 634, 10)      910       
                                                                 
 conv2d_6 (Conv2D)           (None, 558, 632, 10)      9

In [13]:
model = tf.keras.Sequential([])
for i in model_arch:
    if i.lower == "conv":
        model.add(tf.keras.layers.Conv2D(filters= filters_size, kernel_size=kernal, activation = "relu",input_shape = img_shape ))
        print(i)
    elif i.lower == "max":
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))  
        print(i)
    elif i.lower == "flatten":
        model.add(tf.keras.layers.Flatten())
        print(i)
    elif i.lower == "dnn":
        model.add(tf.keras.layers.Dense(dense_layer))
        print(i)
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.